In [ ]:
#!/usr/bin/env python

import itertools

import numpy
import pandas
import pickle
import random
import scipy.stats

from matplotlib import pyplot

%matplotlib inline

In [ ]:
def get_ratios(mut_effect_dict, gt_fit_pool):
    ns = []
    ranges = []
    stddevs = []
    effects_list = []
    mutations = []
    all_effects_list = []
    for pos in range(N):
        for nt1, nt2 in itertools.permutations('AT', 2):
            mut = (pos, nt1, nt2)
            mut_effects = []
            for x in mut_effect_dict[mut]:
                mut_effects.append(x[3] - x[2])
            all_effects_list.extend(mut_effects)
            mut_effects = numpy.array(mut_effects)
            ns.append(mut_effects.shape[0])
            mutations.append(mut)
            ranges.append(mut_effects.max() - mut_effects.min())
            stddevs.append(numpy.std(mut_effects))
            effects_list.append(mut_effects)
    all_effects_arr = numpy.array(all_effects_list)
    null_ranges = []
    null_stddevs = []
    null_effects_list = []
    for n in ns:
        null_effects = numpy.array([
            x[1] - x[0] for x in
            numpy.random.choice(gt_fit_pool, size=(n, 2), replace=True)
        ])
        null_ranges.append(null_effects.max() - null_effects.min())
        null_stddevs.append(numpy.std(null_effects))
        null_effects_list.append(null_effects)
    ranges = numpy.array(ranges)
    null_ranges = numpy.array(null_ranges)
    stddevs = numpy.array(stddevs)
    null_stddevs = numpy.array(null_stddevs)
    return ranges, null_ranges, stddevs, null_stddevs

In [ ]:
in_dir_all = '../01_fitness/'
out_dir = './plots/'
N = 16

In [ ]:
level_list, ii_range_list, ii_stddev_list = [], [], []
bin_len = (0.05, 0.002)
def get_bins(arr, bin_len):
    return numpy.arange(arr.min(), arr.max()+bin_len, bin_len)
iid_f = open('sigma_iid.list', 'w')
for level in numpy.arange(16):
    dirx = f'nlevel_{level:02d}'
    mut_effect_dict = pickle.load(open(f'{in_dir_all}{dirx}/mut_effect_dict.pkl', 'rb'))
    gt_fit_list = pickle.load(open(f'{in_dir_all}{dirx}/gt_logfit_list.pkl', 'rb'))
    gt_fit_pool = numpy.array(gt_fit_list)
    numpy.random.seed(4032)
    ranges, null_ranges, stddevs, null_stddevs = get_ratios(mut_effect_dict, gt_fit_pool)
    mean_range = numpy.mean(ranges)
    mean_null_range = numpy.mean(null_ranges)
    mean_ii_range = numpy.mean(ranges / null_ranges)
    mean_stddev = numpy.mean(stddevs)
    mean_null_stddev = numpy.mean(null_stddevs)
    mean_ii_stddev = numpy.mean(stddevs / null_stddevs)
    level_list.append(level)
    ii_range_list.append(ranges / null_ranges)
    ii_stddev_list.append(stddevs / null_stddevs)
    print(f'level: {level:d}\trange: {mean_range:.2f}\tnull_range: {mean_null_range:.2f}\tii(range): {mean_ii_range:.2f}'
          f'\tstddev: {mean_stddev:.2f}\tnull_stddev: {mean_null_stddev:.2f}\tii(stddev): {mean_ii_stddev:.2f}');
    print(f'{level:d}\t{mean_ii_stddev:.2f}', file=iid_f);